In [4]:
import numpy as np
import pyopencl as cl

# Создание контекста и командной очереди для OpenCL
context = cl.create_some_context()
queue = cl.CommandQueue(context)

# Ограничения
limit_Nmax = 1e7  # Максимальное количество точек
limit_a = 1e6  # Максимальное значение радиуса круга
a = 100  # Начальный радиус круга

# Код ядра OpenCL для метода Монте-Карло
kernel_code = """
__kernel void monte_carlo(
    const int num_points,
    __global const float* random_x,
    __global const float* random_y,
    __global int* count) {
    
    int i = get_global_id(0);
    if (i >= num_points)
        return;

    float x = random_x[i];
    float y = random_y[i];
    if (x * x + y * y <= 1.0f)
        atomic_add(count, 1);
}
"""

# Компиляция программы OpenCL
program = cl.Program(context, kernel_code).build()

# Функция для вычисления Pi методом Монте-Карло
def compute_pi():
    global a
    pi_estimate = 0.0

    while a < limit_a:
        Nmax = a
        while Nmax <= limit_Nmax:
            num_points = int(Nmax)

            # Генерация случайных точек
            random_x = np.random.rand(num_points).astype(np.float32)
            random_y = np.random.rand(num_points).astype(np.float32)
            count = np.zeros(1).astype(np.int32)

            # Создание буферов
            buffer = cl.mem_flags
            random_x_buf = cl.Buffer(context, buffer.READ_ONLY | buffer.COPY_HOST_PTR, hostbuf=random_x)
            random_y_buf = cl.Buffer(context, buffer.READ_ONLY | buffer.COPY_HOST_PTR, hostbuf=random_y)
            count_buf = cl.Buffer(context, buffer.READ_WRITE | buffer.COPY_HOST_PTR, hostbuf=count)

            # Настройка и запуск ядра
            monte_carlo_kernel = program.monte_carlo
            monte_carlo_kernel.set_args(np.int32(num_points), random_x_buf, random_y_buf, count_buf)

            global_size = (num_points,)
            cl.enqueue_nd_range_kernel(queue, monte_carlo_kernel, global_size, None).wait()
            cl.enqueue_copy(queue, count, count_buf).wait()

            # Обновление значений для следующей итерации
            Nmax *= 2
            a *= 2

            # Вычисление значения Pi
            pi_estimate = 4.0 * count[0] / num_points
            print(f"Для a = {a/2} и Nmax = {Nmax/2}, приближенное значение Pi: {pi_estimate}")

    return pi_estimate

# Запуск вычисления
Pi = compute_pi()
print(f"Окончательное приближенное значение Pi: {Pi}")


Для a = 100.0 и Nmax = 100.0, приближенное значение Pi: 3.04
Для a = 200.0 и Nmax = 200.0, приближенное значение Pi: 3.2
Для a = 400.0 и Nmax = 400.0, приближенное значение Pi: 3.13
Для a = 800.0 и Nmax = 800.0, приближенное значение Pi: 3.105
Для a = 1600.0 и Nmax = 1600.0, приближенное значение Pi: 3.2225
Для a = 3200.0 и Nmax = 3200.0, приближенное значение Pi: 3.0725
Для a = 6400.0 и Nmax = 6400.0, приближенное значение Pi: 3.13875
Для a = 12800.0 и Nmax = 12800.0, приближенное значение Pi: 3.1475
Для a = 25600.0 и Nmax = 25600.0, приближенное значение Pi: 3.13265625
Для a = 51200.0 и Nmax = 51200.0, приближенное значение Pi: 3.13875
Для a = 102400.0 и Nmax = 102400.0, приближенное значение Pi: 3.13953125
Для a = 204800.0 и Nmax = 204800.0, приближенное значение Pi: 3.1383984375
Для a = 409600.0 и Nmax = 409600.0, приближенное значение Pi: 3.1440625
Для a = 819200.0 и Nmax = 819200.0, приближенное значение Pi: 3.1408447265625
Для a = 1638400.0 и Nmax = 1638400.0, приближенное значе